In [2]:
import hail as hl

In [3]:
hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'gnomad',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })

2022-08-29 02:58:16 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://4010d8f3bc00:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /home/jupyter/hail-20220829-0258-0.2.97-937922d7f46c.log


In [4]:
bigdata = hl.vds.read_vds('gs://gnomad/v4.0/raw/exomes/gnomad_v4.0.vds')

In [5]:
bigdata.n_samples()

955359

In [ ]:
acta_intervals = hl.experimental.get_gene_intervals(transcript_ids=['ENST00000366684'], reference_genome='GRCh38')

In [ ]:
acta_intervals

In [ ]:
smalldata = hl.vds.filter_intervals(bigdata, acta_intervals, split_reference_blocks=False, keep=True)

In [ ]:
smalldata.variant_data.count()

In [ ]:
#smalldata = hl.vds.split_multi(smalldata)
#smalldata.variant_data.count()

In [ ]:
acta1_mt = hl.vds.to_dense_mt(smalldata)

In [ ]:
acta1_mt.describe()

In [ ]:
smalldata.variant_data.show(10,10)

In [ ]:
# annotate with frequencies
mt = hl.variant_qc(

In [6]:
TRANSCRIPTS = [
    "ENST00000366684", # ACTA1
    "ENST00000371953",  # PTEN
    "ENST00000298910", # LRRK2
    "ENST00000320868", # HBA1
    "ENST00000359125", # KCNQ2
    "ENST00000393087", # SERPINA1
    "ENST00000281708",  # FBXW7
    "ENST00000640769", # RTTN
    "ENST00000270139", # IFNAR1
    "ENST00000262030" # ATP5F1B
]

In [ ]:
TRANSCRIPTS

In [ ]:
# find genomics coordinates of target transcripts and filter vds
target_intervals = hl.experimental.get_gene_intervals(transcript_ids=TRANSCRIPTS, reference_genome='GRCh38')
small_vds = hl.vds.filter_intervals(bigdata, target_intervals, split_reference_blocks=False, keep=True)

In [ ]:
# densify as a MatrixTable
mt = hl.vds.to_dense_mt(small_vds)

In [ ]:
# get alleles frequencies
mt = mt.transmute_entries(GT=hl.experimental.lgt_to_gt(mt.LGT, mt.LA))
mt = hl.variant_qc(mt)

In [ ]:
# keep only the rows and write down
ht = mt.rows()

In [ ]:
ht.describe()

In [ ]:
ht.write("small_ht", overwrite=True)

In [ ]:
out_file = open("transcripts.json", "w")
json.dump(TRANSCRIPTS, out_file, indent = 6)
out_file.close()

In [ ]:
import json

In [ ]:
transcripts_file = open("transcripts.json", "r")
TR = json.load(transcripts_file)

In [ ]:
TR

In [ ]:
ht.count()

In [10]:
if (1<2):
    print('toto')

toto


In [ ]:
GENCODE = 'gencode.v39.basic.annotation.gtf.bgz'
target_intervals = hl.experimental.get_gene_intervals(transcript_ids=TRANSCRIPTS, reference_genome='GRCh38', gtf_file=GENCODE)

2022-08-26 07:57:24 Hail: INFO: Reading table without type imputation0 + 1) / 1]
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type int32 (user-supplied)
  Loading field 'f4' as type int32 (user-supplied)
  Loading field 'f5' as type float64 (user-supplied)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type int32 (user-supplied)
  Loading field 'f8' as type str (not specified)
2022-08-26 07:57:37 Hail: INFO: wrote table with 1905483 rows in 12 partitions to /tmp/uElheSesW6hLJrfJ4xBEEx
2022-08-26 07:57:41 Hail: INFO: Ordering unsorted dataset with network shuffle2]


In [ ]:
small_vds = hl.vds.filter_intervals(big_vds, target_intervals, split_reference_blocks=False, keep=True)